## Analysing genetic effect models using symbolic computations

In addition to simulation and data analysis, genetic effects models can also be approached analytically with the aid of symbolic computation systems (doi:[10.1080/10705511.2024.2325122](https://doi.org/10.1080/10705511.2024.2325122), doi:[10.32614/rj-2023-090](https://doi.org/10.32614/rj-2023-090)). Here we use [SymPy](https://www.sympy.org) to encode a model symbolically and then explore it mathematically.

In [452]:
# import required functions
from sympy.stats import P, E, variance, covariance, correlation, Variance, Covariance
from sympy import Symbol, symbols, sqrt, simplify, Matrix

### Model setup

Encode additive genetic model with tagged and untagged genetic variance, plus environmental variance.

In [363]:
Va_tagged = Symbol('V_at', positive = True)
Va_untagged = Symbol('V_au', positive = True)
Ve = Symbol('V_e', positive = True)

The phenotypic variance is therefore

In [364]:
Va_tagged + Va_untagged + Ve

V_at + V_au + V_e

Create maternal and paternal / tagged and untagged genetic effects that are normally distributed with the specified standard deviations.

In [ ]:
# maternal and paternal tagged and untagged genetic effects

# maternal tagged
A_mt = Normal("A_mt", 0, sqrt(Va_tagged))
# maternal untagged
A_mu = Normal("A_m_u", 0, sqrt(Va_untagged))
# paternal tagged
A_pt = Normal("A_pt", 0, sqrt(Va_tagged))
# paternal untagged
A_pu = Normal("A_pu", 0, sqrt(Va_untagged))

The total genetic effects for each parent are the sums of the tagged and untagged effects

In [365]:
A_m = A_mt + A_mu
A_p = A_pt + A_pu

Offspring genetic effects have two parts: the average of the parental effects plus segregation variance (half of $V_\mathrm{A}$).

In [355]:
# offspring
# segregation variance (tagged and untagged)
A_ots = Normal("A_ots", 0, sqrt(Va_tagged/2))
A_ous = Normal("A_ous", 0, sqrt(Va_untagged/2))
# average of parents plus segregation
A_ot = (A_mt + A_pt)/2 + A_ots
A_ou = (A_mu + A_pu)/2 + A_ous

A_o = A_ot + A_ou

Maternal, paternal, and offspring environment effects (assume uncorrelated in this model.)

In [ ]:
# environment
E_m = Normal("E_m", 0, sqrt(Ve))
E_p = Normal("E_p", 0, sqrt(Ve))
E_o = Normal("E_o", 0, sqrt(Ve))


Add all effects together to make phenotypes.

In [356]:
# phenotypes
P_m = A_m + E_m
P_p = A_p + E_p
P_o = A_o + E_o

Evaluate the phenotypic variances to check that the model was defined as expected.

In [360]:
variance(P_m)

V_at + V_au + V_e

In [ ]:
variance(P_p) 

V_at + V_au + V_e

In [370]:
variance(P_o)

V_at + V_au + V_e

Finally, define additional symbols summarising variances:

In [397]:
Vp = Symbol('V_p', positive = True)
h2_snp = Symbol('h^2_SNP', positive = True)
h2 = Symbol('h^2', positive = True)

### Exploring full and partial correlations

As a basic exploration, we will look at the correlation between maternal and offspring phenotypes. The covariance is:

In [376]:
covariance(P_m, P_o)

V_at/2 + V_au/2

while the correlation is

In [454]:
mother_offspring_rp = correlation(P_m, P_o)
mother_offspring_rp

(V_at/2 + V_au/2)/(V_at + V_au + V_e)

If there is no untagged genetic variance (the tagged variance accounts for all of the additive genetic variance), the expected correlations is:

In [455]:
mother_offspring_rp_Vau0 = mother_offspring_rp.subs(Va_untagged, 0)
mother_offspring_rp_Vau0

V_at/(2*(V_at + V_e))

Express correlation in terms of phenotypic variance $V_p$ and SNP heritability $h^2_{SNP}$

In [398]:
mother_offspring_rp_Vau0_vp = mother_offspring_rp_Vau0.subs(Va_tagged + Ve, Vp)
mother_offspring_rp_Vau0_vp

V_at/(2*V_p)

In [400]:
mother_offspring_rp_Vau0_vp.subs(Va_tagged / Vp, h2_snp)

h^2_SNP/2

Examine partial correlation between maternal and offspring phenotypes after controlling for tagged genetic effects. First, create a covariance matrix $\Sigma$ for the maternal and offspring phenotypes and tagged genetic effects:

In [441]:
M = [P_m, P_o, A_mt, A_ot]
Sigma = Matrix(len(M), len(M), lambda i,j: covariance(M[i], M[j]))
Sigma

Matrix([
[V_at + V_au + V_e,   V_at/2 + V_au/2,   V_at, V_at/2],
[  V_at/2 + V_au/2, V_at + V_au + V_e, V_at/2,   V_at],
[             V_at,            V_at/2,   V_at, V_at/2],
[           V_at/2,              V_at, V_at/2,   V_at]])

Calculate the precision matrix $\Omega$ as the inverse of $\Sigma$

In [442]:
Omega = Sigma.inv()
Omega

Matrix([
[ (4*V_au + 4*V_e)/(3*V_au**2 + 8*V_au*V_e + 4*V_e**2),           -2*V_au/(3*V_au**2 + 8*V_au*V_e + 4*V_e**2),                                                                    (-4*V_au - 4*V_e)/(3*V_au**2 + 8*V_au*V_e + 4*V_e**2),                                                                               2*V_au/(3*V_au**2 + 8*V_au*V_e + 4*V_e**2)],
[          -2*V_au/(3*V_au**2 + 8*V_au*V_e + 4*V_e**2),  (4*V_au + 4*V_e)/(3*V_au**2 + 8*V_au*V_e + 4*V_e**2),                                                                               2*V_au/(3*V_au**2 + 8*V_au*V_e + 4*V_e**2),                                                                    (-4*V_au - 4*V_e)/(3*V_au**2 + 8*V_au*V_e + 4*V_e**2)],
[(-4*V_au - 4*V_e)/(3*V_au**2 + 8*V_au*V_e + 4*V_e**2),            2*V_au/(3*V_au**2 + 8*V_au*V_e + 4*V_e**2), (12*V_at*V_au + 12*V_at*V_e + 12*V_au**2 + 32*V_au*V_e + 16*V_e**2)/(9*V_at*V_au**2 + 24*V_at*V_au*V_e + 12*V_at*V_e**2),                 (-6*V_at*V_au - 6*V_au**2 - 16

The partial correlation between the maternal and offspring phenotypes ($P_m$ and $P_o$) accounting for the tagged genetic effects ($A_{m_t}$, $A_{o_t}$) listed $\mathbf{M}$ is

$$
\rho_{P_m P_o \cdot \mathbf{M}} = -\frac{p_{P_mP_o}}{\sqrt{p_{P_mP_m}p_{P_oP_o}}}
$$

$P_m$ and $P_o$ are the first and second elements in $\Omega$, thus

In [482]:
def partial_corr(Omega, i, j):
  rho = (-Omega[i,j] / sqrt(Omega[i,i] * Omega[j,j]))
  return(rho)

In [483]:
mother_offspring_pheno_partial_corr = partial_corr(Omega, 0, 1)
mother_offspring_pheno_partial_corr.simplify()

V_au/(2*(V_au + V_e))

and the partial correlation would be positive

In [484]:
mother_offspring_pheno_partial_corr > 0

True

If instead the tagged genetic variance $V_{a_t}$ is 0, then the partial correlation between maternal and offspring phenotpyes controlling for tagged genetic effects is:

In [450]:
mother_offspring_pheno_partial_corr.subs(Va_untagged, 0) == 0

True

otherwise it is proportional to $V_{a_u}$, which could appear to be an nurturing effect. Similarly, the correlation between the offspring phenotype and the maternal tagged genetic effect is:

In [486]:
correlation(P_o, A_mt)

sqrt(V_at)/(2*sqrt(V_at + V_au + V_e))

while the partial correlation (controlling for maternal phenotype and offspring tagged genetic effect) is negative:

In [487]:
# offspring phenotype = 2nd element, maternal tagged genetic effect = 3rd element of Omega
partial_corr(Omega, 1, 2) < 0

True

### Conclusion

Using symbolic computation to explore genetic models eases the gap between writing code and mathematical analysis.

### Appendix

Instead of building the covariance matrix $\Sigma$ using a `lambda` constructor looping over the indices of the matrix, it can also be calculated as the expectation of the multiplication of the transposed matrix with itself:

In [477]:
# turn the list of variables into a matrix
X = Matrix([M])
# evaluating expection (E()) returns calculation in terms of variables variances
S = E(X.T * X)
# invert the matrix
O = S.inv()
# test that this is the same as the value of Omega
O == Omega

True